# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [28]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

data_path = 'data/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [29]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
df['Cabin'] = df['Cabin'].fillna('None')
df['Fare'] = df['Fare'].fillna(df['Fare'].mean())

cabin_mean_fare = df.groupby(['Cabin'])['Fare'].mean().reset_index()
cabin_median_fare = df.groupby(['Cabin'])['Fare'].median().reset_index()
cabin_mode_fare = df.groupby(['Cabin'])['Fare'].apply(lambda x: x.mode()[0]).reset_index()

data = pd.merge(cabin_mean_fare, cabin_median_fare, how = 'left', on = ['Cabin'])
data = pd.merge(data, cabin_mode_fare, how = 'left', on = ['Cabin'])
data.columns = ['Cabin', 'cabin_mean_fare', 'cabin_median_fare', 'cabin_mode_fare']
print(data)

    Cabin  cabin_mean_fare  cabin_median_fare  cabin_mode_fare
0     A10        40.125000           40.12500          40.1250
1     A14        52.000000           52.00000          52.0000
2     A16        39.600000           39.60000          39.6000
3     A19        26.000000           26.00000          26.0000
4     A20        56.929200           56.92920          56.9292
..    ...              ...                ...              ...
143   F38         7.750000            7.75000           7.7500
144    F4        39.000000           39.00000          39.0000
145    G6        13.581250           13.58125          10.4625
146  None        19.157325           10.50000           8.0500
147     T        35.500000           35.50000          35.5000

[148 rows x 4 columns]


In [30]:
df = pd.merge(df, data, how = 'left', on = ['Cabin'])
df = df.drop(['Cabin'], axis = 1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,cabin_mean_fare,cabin_median_fare,cabin_mode_fare
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,19.157325,10.5000,8.0500
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,71.283300,71.2833,71.2833
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,19.157325,10.5000,8.0500
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,53.100000,53.1000,53.1000
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,19.157325,10.5000,8.0500


In [31]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

8 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'cabin_mean_fare', 'cabin_median_fare', 'cabin_mode_fare']



,Pclass,Age,SibSp,Parch,Fare,cabin_mean_fare,cabin_median_fare,cabin_mode_fare
0,3,22.0,1,0,7.2500,19.157325,10.5000,8.0500
1,1,38.0,1,0,71.2833,71.283300,71.2833,71.2833
2,3,26.0,0,0,7.9250,19.157325,10.5000,8.0500
3,1,35.0,1,0,53.1000,53.100000,53.1000,53.1000
4,3,35.0,0,0,8.0500,19.157325,10.5000,8.0500


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [33]:
# 原始特徵 + 邏輯斯迴歸
df_minus = df.drop(['cabin_mean_fare', 'cabin_median_fare', 'cabin_mode_fare'], axis = 1)
train_x = MMEncoder.fit_transform(df_minus)
estimator = LogisticRegression()
cross_val_score(estimator, train_x, train_Y, cv = 5).mean()

0.70041428661101

In [34]:
# 新特徵 + 邏輯斯迴歸
train_x = MMEncoder.fit_transform(df)
estimator = LogisticRegression()
cross_val_score(estimator, train_x, train_Y, cv = 5).mean()

0.701537882116628

In [ ]:
# 由上可見，新特徵下的分數略微上升